# **CHAPTER 2. Phylogenetic analysis**

Import all the modules needed for analysis

In [5]:
import csv
import pandas as pd

Process `Annotation/LSTINFO-.lst` file to leave there only those 62% of genomes that have 4 commonly shared genes!

In [2]:
%run scripts/process_LSTINFO.py Coregenome/PersGenome_PanGenome-LeMy.All.prt-clust-0.8-mode1.lst-all_0.62.lst Annotation/LSTINFO-.lst Annotation/filtered_LSTINFO-.lst

Filtered file created: Annotation/filtered_LSTINFO-.lst


> Disclaimer: the command below was run on DSTU's server

Lifehack: instead of running MAFFT by ourselves, we can still run `PanACoTA`!

In [8]:
! PanACoTA align -c Coregenome/PersGenome_PanGenome-LeMy.All.prt-clust-0.8-mode1.lst-all_0.62.lst -l Annotation/filtered_LSTINFO-.lst -n LeMy -d Annotation/ -o Alignment

  * [2025-01-31 16:10:50] : INFO  PanACoTA version 1.4.0
  * [2025-01-31 16:10:50] : INFO  Command used
 	 > PanACoTA align -c Coregenome/PersGenome_PanGenome-LeMy.All.prt-clust-0.8-mode1.lst-all_0.62.lst -l Annotation/15_19_filtered_LSTINFO-.lst -n LeMy -d Annotation/ -o Alignment_15_19
  * [2025-01-31 16:10:50] : INFO  Found 20 genomes.
  * [2025-01-31 16:10:50] : INFO  Reading PersGenome and constructing lists of missing genomes in each family.
  * [2025-01-31 16:10:50] : INFO  Getting all persistent proteins and classify by strain.
  * [2025-01-31 16:10:50] : INFO  Extracting proteins and genes from all genomes
Extraction:████████████████ 20/20 (100%) - Elapsed Time: 0:00:00 Time:  0:00:00
  * [2025-01-31 16:10:50] : INFO  Starting alignment of all families: protein alignment, back-translation to nucleotides, and add missing genomes in the family
Alignment: ██████████████████████████████ 4/4 (100%) - Elapsed Time: 0:00:03 -   * [2025-01-31 16:10:54] : INFO  Concatenating all nucl a

Now it is time to work with MSAs a little bit<br>
First, create a directory where to store them

In [43]:
! mkdir Alignment/MSAs/

Let's take a look at the current state of MSAs

In [1]:
! head -1 Alignment/Align-LeMy/LeMy-mafft-align.314.aln

>LeMy.0125.00001.0001i_00011 1491 nuoM | NADH-quinone oxidoreductase subunit M | 7.1.1.- | similar to AA sequence:UniProtKB:P0AFE8 | COG:COG1008


TOTAL MESS! We must restore the normal state of seqs names

Create a name mapping for further renaming

In [44]:
# Step 1: Load the TSV file into a dictionary
tsv_file = 'Annotation/filtered_LSTINFO-.lst'
name_mapping = {}

with open(tsv_file, 'r') as file:
    reader = csv.DictReader(file, delimiter='\t')
    for row in reader:
        # Mapping the gembase_name to orig_name without the .fasta extension
        gembase_name = row['gembase_name']
        orig_name = row['orig_name'].replace('.fasta', '')  # Remove .fasta extension
        name_mapping[gembase_name] = orig_name

The function below will take old MSA file with messy seqs names and output MSA with normal seqs names!

In [ ]:
def process_msa(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        lines = infile.readlines()
        
        for line in lines:
            if line.startswith('>'):
                # Extract the gembase_name from the FASTA header (before the first 3rd dot)
                #if output_file == 'Alignment/MSAs/concat.aln':
                    #parts = line.strip().split(' ')[0][1:]
                    #gembase_name = '.'.join(parts.split('.'))
                #else:
                parts = line.split(' ')[0][1:]  # Extract the part before the first space and remove '>'
                gembase_name = '.'.join(parts.split('.')[:3])
                
                # Look up the corresponding NC_XXX.2 name from the dictionary
                if gembase_name in name_mapping:
                    new_header = f">{name_mapping[gembase_name]}\n"
                    outfile.write(new_header)
                else:
                    # If not found in the mapping, keep the original header
                    #outfile.write(line)
                    continue
            elif gembase_name in name_mapping:
                # Write sequence lines as they are
                outfile.write(line)

    print("FASTA file has been renamed successfully.")

Rename `Alignment/Align-LeMy/LeMy-mafft-align.314.aln`. As we remember it was `NADH-quinone oxidoreductase subunit M`. So, give the new file this name!

In [46]:
# Step 2: Process the FASTA file and rename headers
fasta_file = 'Alignment/Align-LeMy/LeMy-mafft-align.314.aln'
output_fasta_file = 'Alignment/MSAs/NADH-quinone_oxidoreductase_subunit_M.aln'

process_msa(fasta_file, output_fasta_file)

FASTA file has been renamed successfully.


Now it is time for `Alignment/Align-LeMy/LeMy-mafft-align.368.aln`

In [2]:
! head -1 Alignment/Align-LeMy/LeMy-mafft-align.368.aln

>LeMy.0125.00001.0001i_00051 270 nuoK | NADH-quinone oxidoreductase subunit K | 7.1.1.- | similar to AA sequence:UniProtKB:A1B482 | COG:COG0713


Give it a `NADH-quinone oxidoreductase subunit K` name!

In [47]:
# Step 2: Process the FASTA file and rename headers
fasta_file = 'Alignment/Align-LeMy/LeMy-mafft-align.368.aln'
output_fasta_file = 'Alignment/MSAs/NADH-quinone_oxidoreductase_subunit_K.aln'

process_msa(fasta_file, output_fasta_file)

FASTA file has been renamed successfully.


Next comes `Alignment/Align-LeMy/LeMy-mafft-align.1352.aln`

In [3]:
! head -1 Alignment/Align-LeMy/LeMy-mafft-align.1352.aln

>LeMy.0125.00001.0001b_00055 1167 fbcH | Cytochrome b/c1 | NA | similar to AA sequence:UniProtKB:P51131 | COG:COG1290


Okay, the name is `Cytochrome b/c1`!

In [48]:
# Step 2: Process the FASTA file and rename headers
fasta_file = 'Alignment/Align-LeMy/LeMy-mafft-align.1352.aln'
output_fasta_file = 'Alignment/MSAs/Cytochrome_b_c1.aln'

process_msa(fasta_file, output_fasta_file)

FASTA file has been renamed successfully.


Finally, `Alignment/Align-LeMy/LeMy-mafft-align.1664.aln`

In [4]:
! head -1 Alignment/Align-LeMy/LeMy-mafft-align.1664.aln

>LeMy.0125.00001.0001i_00012 774 atpB | ATP synthase subunit a | NA | similar to AA sequence:UniProtKB:O05330 | COG:COG0356


And its name is `ATP synthase subunit a`

In [49]:
# Step 2: Process the FASTA file and rename headers
fasta_file = 'Alignment/Align-LeMy/LeMy-mafft-align.1664.aln'
output_fasta_file = 'Alignment/MSAs/ATP_synthase_subunit_a.aln'

process_msa(fasta_file, output_fasta_file)

FASTA file has been renamed successfully.


When the MSAs are ready it is time to trim them all!!!

First, create a directory to store trimmed MSAs

In [ ]:
! mkdir Alignment/trimmed_MSAs/

Then run a bash loop with `trimAl` in it on them

In [54]:
%%bash

for msa in Alignment/MSAs/*.aln
do trimal -in $msa -out Alignment/trimmed_MSAs/$(basename "$msa" .aln)_trim.fa -automated1
done

We've got the trimmed MSAs! Now what? MODEL-FINDER!

First, create a directory to store ModelFinder log files

In [65]:
! mkdir model-finder/

Then run a bash loop with `ModelFinder` in it on trimmed MSAs!

In [ ]:
%%bash

for msa in Alignment/trimmed_MSAs/*_trim.fa
do 
    iqtree2 -m MFP -s $msa --prefix model-finder/$(basename "$msa" _trim.fa) -T AUTO
done

Get the best fit model for `ATP synthase subunit a`

In [67]:
! head -42 model-finder/ATP_synthase_subunit_a.iqtree | tail -6

Best-fit model according to BIC: mtZOA+I+G4

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
mtZOA+I+G4        -1699.484    3452.968 - 5.44e-05    3459.542 +   0.0616    3548.898 +    0.501


Get the best fit model for `Cytochrome b/c1`

In [68]:
! head -42 model-finder/Cytochrome_b_c1.iqtree | tail -6

Best-fit model according to BIC: mtZOA+I+R2

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
mtZOA+I+R2        -2282.265    4624.529 -  0.00112    4629.844 +    0.132    4742.813 +    0.968


Get the best fit model for `NADH-quinone oxidoreductase subunit K`

In [69]:
! head -42 model-finder/NADH-quinone_oxidoreductase_subunit_K.iqtree | tail -6

Best-fit model according to BIC: mtMet+G4

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
mtMet+G4           -439.903     943.806 -  0.00606     981.520 +    0.227    1023.442 +    0.186


Get the best fit model for `NADH-quinone oxidoreductase subunit M`

In [70]:
! head -42 model-finder/NADH-quinone_oxidoreductase_subunit_M.iqtree | tail -6

Best-fit model according to BIC: mtZOA+G4

List of models sorted by BIC scores: 

Model                  LogL         AIC      w-AIC        AICc     w-AICc         BIC      w-BIC
mtZOA+G4          -3022.927    6097.854 - 2.74e-13    6100.919 - 8.08e-12    6206.642 +    0.609


At last... We have best fit models for all of our 4 genes... It's time to launch `IQ-TREE`!

First, create a directory to store the trees

In [71]:
! mkdir tree/

Then run a bash loop with `IQ-TREE` in it on trimmed MSAs

In [ ]:
%%bash

for msa in Alignment/trimmed_MSAs/*_trim.fa
do 
    if [[ "$msa" == "ATP_synthase_subunit_a_trim.fa" ]]; then
        model="mtZOA+I+G4"
    elif [[ "$msa" == "Cytochrome_b_c1_trim.fa" ]]; then
        model="mtZOA+I+R2"
    elif [[ "$msa" == "NADH-quinone_oxidoreductase_subunit_K_trim.fa" ]]; then
        model="mtMet+G4"
    elif [[ "$msa" == "NADH-quinone_oxidoreductase_subunit_M_trim.fa" ]]; then
        model="mtZOA+G4"
    fi

    iqtree2 -s "$msa" -m "$model" -pre tree/$(basename "$msa" _trim.fa)_ufb -bb 10000 -nt AUTO
done

Actually that's all!<br>
But now we'll fetch metadata on _Leotiomycetes_ fungi from NCBI RefSeq<br>
It will be used to annotate the trees in `ggtree`!

First, we need the list of accession numbers

In [1]:
! esearch -db nucleotide \
    -query '("Leotiomycetes"[Organism] OR Leotiomycetes[All Fields]) AND srcdb_refseq[PROP] AND (fungi[filter] AND mitochondrion[filter])' \
    | efetch -format acc > data/accession_numbers.txt

Now create a directory where to store metadat

In [4]:
! mkdir metadata/

Fetch metadata!

In [2]:
! Phyloki --fetch_metadata -email ivpopov@donstu.ru -i data/accession_numbers.txt -o metadata/raw_metadata.tsv

Metadata retrieval complete.
File saved to metadata/raw_metadata.tsv


This function will clean the `Year` column!

In [3]:
def clean_year(input_file, output_file):
    """
    Clean the 'Year' column in a metadata .tsv file to extract only the last 4 digits.

    Args:
        input_file (str): Path to the input .tsv file.
        output_file (str): Path to save the cleaned .tsv file.
    """
    # Load the .tsv file into a DataFrame
    df = pd.read_csv(input_file, sep="\t")

    # Extract the last 4 digits of the 'Year' column
    df['Year'] = df['Year'].apply(lambda x: str(x)[-4:] if pd.notnull(x) else 'ND')

    # Save the updated DataFrame to a new .tsv file
    df.to_csv(output_file, sep="\t", index=False)

    print(f"The 'Year' column has been cleaned.\nFile saved to {output_file}")

Run the `clean_year()` function!

In [6]:
clean_year('metadata/raw_metadata.tsv', 'metadata/metadata.tsv')

The 'Year' column has been cleaned.
File saved to metadata/metadata.tsv


Now please proceed to the `04_ggtree_journal.R` to visualize the phylogenetic trees

Additionally, `NADH-quinone oxidoreductase subunit K` gene MSA will be visualized in `05_ggmsa_journal.R`